In [1]:
from os import path
import numpy as np
import pandas as pd
from IPython.display import display

if not path.exists("data/raw/StreamingHistory.csv"):
    streaming_history_0_df = pd.read_json("data/raw/StreamingHistory0.json")
    streaming_history_1_df = pd.read_json("data/raw/StreamingHistory1.json")
    streaming_history_2_df = pd.read_json("data/raw/StreamingHistory2.json")

    stream_history_df = pd.concat([streaming_history_0_df, streaming_history_1_df, streaming_history_2_df])
    stream_history_df.to_csv("data/raw/StreamingHistory.csv")
else:
    stream_history_df = pd.read_csv("data/raw/StreamingHistory.csv")
    stream_history_df.drop(stream_history_df.columns[0], axis=1, inplace=True)

print(stream_history_df.shape)
stream_history_df.head()

(26513, 4)


/home/jcroderman/.pyenv/versions/3.7.5/lib/python3.7/site-packages/pandas/compat/__init__.py:120: UserWarning: Could not import the lzma module. Your installed Python is incomplete. Attempting to use lzma compression will result in a RuntimeError.
  warnings.warn(msg)


,endTime,artistName,trackName,msPlayed
0,2019-10-30 00:15,Greensky Bluegrass,Time > Breathe Reprise >,129948
1,2019-10-30 00:34,Greensky Bluegrass,Time > Breathe Reprise >,1360
2,2019-10-30 00:52,Matthew Barber,Sleep Please Come to Me,172520
3,2019-10-30 00:58,Matthew Barber,Somebody Sometime,262760
4,2019-10-30 00:58,Matthew Barber,Our Voices,30951


## Cleaning Up the Data

First, fix the date data

In [2]:
from datetime import timedelta

stream_history_df["date"] = pd.to_datetime(stream_history_df["endTime"], format="%Y-%m-%d")
stream_history_df["date"] = stream_history_df["date"].apply(lambda date: date - timedelta(hours = 6))
stream_history_df.drop("endTime", axis=1, inplace=True)
stream_history_df["date"].describe(datetime_is_numeric=True)

count                            26513
mean     2020-05-03 07:57:06.474559744
min                2019-10-29 18:15:00
25%                2020-02-02 15:57:00
50%                2020-04-29 15:40:00
75%                2020-08-13 14:20:00
max                2020-10-30 16:51:00
Name: date, dtype: object

In [3]:
stream_history_df = stream_history_df.rename({
    "artistName": "artist_name",
    "trackName": "track_name",
    "msPlayed": "ms_played"
}, axis=1)

Just for simplicity sake, I'm only going to focus on music listened to since January 1st, 2020

In [4]:
stream_history_df = stream_history_df[~(stream_history_df['date'] < '2020-01-01')]
stream_history_df.date.describe(datetime_is_numeric=True)

count                            21670
mean     2020-06-07 18:13:02.447623680
min                2020-01-01 00:51:00
25%                2020-03-14 10:36:00
50%                2020-06-09 09:57:00
75%                2020-08-28 09:31:45
max                2020-10-30 16:51:00
Name: date, dtype: object

Before I get rid of anything, let's take a look at how long I listened to music before October 30th!

In [5]:
time_streamed_ms = sum(stream_history_df.ms_played)
time_streamed_sec = time_streamed_ms / 1000
time_streamed_min = time_streamed_sec / 60
time_streamed_hour = time_streamed_min / 60
time_streamed_day = time_streamed_hour / 24
print("Days I have listened to Spotify: {:.3f} days".format(time_streamed_day))

Days I have listened to Spotify: 42.687 days


## Consolidating Data

We're about to make a helluva lot of calls to Spotify, so we'll want to consolidate the data to make as few calls as possible. To do this, we'll gather all nonunique tracks

In [6]:
stream_history_consolidated_df = stream_history_df.drop(["date", "ms_played"], axis=1)
stream_history_consolidated_df = stream_history_consolidated_df.groupby(["track_name", "artist_name"]).size().reset_index(name='counts')
print(stream_history_consolidated_df.shape)
stream_history_consolidated_df.head()

(5197, 3)


,track_name,artist_name,counts
0,Lab-Grown Meat: We Grill It,Science Vs,4
1,"""61'st and Rich' It""",Thad Jones,1
2,"""B"" Movie Box Car Blues - Live",The Blues Brothers,3
3,"""I have a Queery about...Being Gay and Greek"" ...",Queer Queeries,2
4,#100 Friends and Blasphemers,Reply All,1


In [7]:
import requests
import re
import time
from urllib.parse import urlencode
token = open('token.txt', 'r').read().strip()

base_url = "https://api.spotify.com/v1/search"
query_obj = {"type": "track,episode", "market": "US", "limit": 1}

def get_spotify_data(track_name, artist_name):
    query_obj["q"] = str.format('"{}" AND "{}"', track_name, artist_name)
    res = requests.get(base_url + "?" + urlencode(query_obj), headers={
        "Authorization": "Bearer " + token
    })
    if res.status_code != 200:
        raise Exception("Spotify call failed, something has gone wrong")
    res_json = res.json()

    track = res_json["tracks"]["items"][0] if len(res_json["tracks"]["items"]) > 0 else None
    episode = res_json["episodes"]["items"][0] if len(res_json["episodes"]["items"]) > 0 else None
    
    time.sleep(0.1) # Remove
    return pd.Series([track, episode])

In [8]:
import time

if not path.exists("data/processed/StreamingHistoryWithData.csv"):
    time_start = time.time()
    stream_history_consolidated_df[["track_data", "episode_data"]] = stream_history_consolidated_df.apply(lambda x: get_spotify_data(x[0], x[1]), axis=1)
    time_elapsed = time.time() - time_start
    print("Time elapsed: {:.3f} seconds".format(time_elapsed)) # Fun fact: last time I ran this it took 32 minutes!
    stream_history_consolidated_df.to_csv("data/processed/StreamingHistoryWithData.csv")
else:
    stream_history_consolidated_df = pd.read_csv("data/processed/StreamingHistoryWithData.csv")
    stream_history_consolidated_df.drop(stream_history_consolidated_df.columns[0], axis=1, inplace=True)

stream_history_consolidated_df.head()

,track_name,artist_name,counts,track_data,episode_data
0,Lab-Grown Meat: We Grill It,Science Vs,4,NaN,{'audio_preview_url': 'https://p.scdn.co/mp3-p...
1,"""61'st and Rich' It""",Thad Jones,1,"{'album': {'album_type': 'album', 'artists': [...",NaN
2,"""B"" Movie Box Car Blues - Live",The Blues Brothers,3,"{'album': {'album_type': 'album', 'artists': [...",NaN
3,"""I have a Queery about...Being Gay and Greek"" ...",Queer Queeries,2,NaN,{'audio_preview_url': 'https://p.scdn.co/mp3-p...
4,#100 Friends and Blasphemers,Reply All,1,NaN,{'audio_preview_url': 'https://p.scdn.co/mp3-p...


### Cleaning the Data
Let's make sure this is all correct...

In [9]:
overloaded_data_df = stream_history_consolidated_df[stream_history_consolidated_df[["track_data", "episode_data"]].notnull().all(1)]
print(overloaded_data_df.shape)
overloaded_data_df.head()

(757, 5)


,track_name,artist_name,counts,track_data,episode_data
56,(Don't Fear) The Reaper,Blue Öyster Cult,3,"{'album': {'album_type': 'album', 'artists': [...",{'audio_preview_url': 'https://p.scdn.co/mp3-p...
61,(I Just) Died In Your Arms,Cutting Crew,1,"{'album': {'album_type': 'album', 'artists': [...",{'audio_preview_url': 'https://p.scdn.co/mp3-p...
62,(I Left My Heart) In San Francisco,Tony Bennett,1,"{'album': {'album_type': 'album', 'artists': [...",{'audio_preview_url': 'https://p.scdn.co/mp3-p...
63,(I Love You) For Sentimental Reasons,Nat King Cole,1,"{'album': {'album_type': 'album', 'artists': [...",{'audio_preview_url': 'https://p.scdn.co/mp3-p...
67,(Sittin' On) the Dock of the Bay,Otis Redding,1,"{'album': {'album_type': 'album', 'artists': [...",{'audio_preview_url': 'https://p.scdn.co/mp3-p...


For the overloaded data, I'm going to have to go through manually and figure out which are which

In [10]:
if not path.exists("data/processed/OverloadedStreamingHistoryWithData.csv"):
    # This means the data has not been scrubbed and needs to be.
    overloaded_data_df.to_csv("data/processed/OverloadedStreamingHistoryWithData.csv");
    print("CLEAN DATA AND RETURN TO MOVE FORWARD")
else:
    overloaded_data_df = pd.read_csv("data/processed/OverloadedStreamingHistoryWithData.csv")
    overloaded_data_df.drop(overloaded_data_df.columns[0], axis=1, inplace=True)
    not_fixed_data = overloaded_data_df.loc[overloaded_data_df["track_data"].notnull() == overloaded_data_df["episode_data"].notnull()]
    if not_fixed_data.shape[0] > 0:
        print(str.format("{} more entries to clean", not_fixed_data.shape[0]))
        display(not_fixed_data)
    else:
        print("Data cleaned! Go ahead and continue!")

Data cleaned! Go ahead and continue!


Now for the missing data...

In [11]:
missing_data_df = stream_history_consolidated_df[stream_history_consolidated_df[["track_data", "episode_data"]].isna().all(1)]
print(missing_data_df.shape)
missing_data_df.head()

(29, 5)


,track_name,artist_name,counts,track_data,episode_data
8,#104 The Case of the Phantom Caller,Reply All,1,NaN,NaN
215,A Night In Tunisia (feat. Rochester Philharmon...,Dizzy Gillespie,1,NaN,NaN
387,Announcement! We’re taking calls and broadcast...,Reply All,1,NaN,NaN
405,"Anything Goes - (From ""Anything Goes"")",Cole Porter,1,NaN,NaN
521,"Battle vs. Kanto Trainer (From ""Pokémon Red an...",Braxton Burks,1,NaN,NaN


For this, I'm going to have to go through and manually find this data

In [12]:
if not path.exists("data/processed/MissingStreamingHistoryWithData.csv"):
    # This means the data has not been retrieved manually yet and needs to be.
    missing_data_df.to_csv("data/processed/MissingStreamingHistoryWithData.csv");
    print("RETRIEVE DATA AND RETURN TO MOVE FORWARD")
else:
    missing_data_df = pd.read_csv("data/processed/MissingStreamingHistoryWithData.csv")
    missing_data_df.drop(missing_data_df.columns[0], axis=1, inplace=True)
    not_fixed_data = missing_data_df.loc[(missing_data_df["track_data"].isna() == missing_data_df["episode_data"].isna())]
    not_fixed_data = not_fixed_data.loc[not_fixed_data["delete"].isna()]
    if not_fixed_data.shape[0] > 0:
        print(str.format("{} more entries to retrieve", not_fixed_data.shape[0]))
        display(not_fixed_data)
    else:
        print("Data cleaned! Go ahead and continue!")

Data cleaned! Go ahead and continue!


Let's put everything back together now!

In [13]:
stream_history_consolidated_df.set_index(["track_name", "artist_name"], inplace=True)
missing_data_df.set_index(["track_name", "artist_name"], inplace=True)
overloaded_data_df.set_index(["track_name", "artist_name"], inplace=True)

stream_history_consolidated_df.update(missing_data_df)
temp_df = pd.concat([stream_history_consolidated_df, overloaded_data_df])
stream_history_consolidated_df = temp_df.loc[~temp_df.index.duplicated(keep='last')]
stream_history_consolidated_df.reset_index(inplace=True)
stream_history_consolidated_df.head()

,track_name,artist_name,counts,track_data,episode_data
0,Lab-Grown Meat: We Grill It,Science Vs,4.0,NaN,{'audio_preview_url': 'https://p.scdn.co/mp3-p...
1,"""61'st and Rich' It""",Thad Jones,1.0,"{'album': {'album_type': 'album', 'artists': [...",NaN
2,"""B"" Movie Box Car Blues - Live",The Blues Brothers,3.0,"{'album': {'album_type': 'album', 'artists': [...",NaN
3,"""I have a Queery about...Being Gay and Greek"" ...",Queer Queeries,2.0,NaN,{'audio_preview_url': 'https://p.scdn.co/mp3-p...
4,#100 Friends and Blasphemers,Reply All,1.0,NaN,{'audio_preview_url': 'https://p.scdn.co/mp3-p...


Now remove the rows that have no data (these are the ones that should be deleted from the `missing_data_df`)

In [14]:
stream_history_consolidated_df = stream_history_consolidated_df.drop(stream_history_consolidated_df[stream_history_consolidated_df[["track_data", "episode_data"]].isna().all(1)].index)
stream_history_consolidated_df[stream_history_consolidated_df[["track_data", "episode_data"]].isna().all(1)].shape

(0, 5)

Let's save this data:

In [15]:
if not path.exists("data/processed/StreamingHistoryWithDataCleaned.csv"):
    overloaded_data_df.to_csv("data/processed/StreamingHistoryWithDataCleaned.csv");

Before we transfer the data back to the main `stream_history_df` dataframe, let's decide what data we want to use:

In [16]:
import ast # Gonna use ast because json has issues with single quotes
import json

track_data = stream_history_consolidated_df[stream_history_consolidated_df["track_data"].notnull()].iloc[0]["track_data"]
episode_data = stream_history_consolidated_df[stream_history_consolidated_df["episode_data"].notnull()].iloc[7]["episode_data"]

def print_nested_keys(obj, offset=""):
    if obj is None:
        return
    if isinstance(obj, list):
        if len(obj) == 0:
            return
        obj = obj[0]
    keys = obj.keys()
    for key in keys:
        print(offset + key)
        try:
            print_nested_keys(obj[key], offset + "  ")
        except:
            pass

print("Track data:")
print_nested_keys(ast.literal_eval(track_data), "  ")

Track data:
  album
    album_type
    artists
      external_urls
        spotify
      href
      id
      name
      type
      uri
    external_urls
      spotify
    href
    id
    images
      height
      url
      width
    name
    release_date
    release_date_precision
    total_tracks
    type
    uri
  artists
    external_urls
      spotify
    href
    id
    name
    type
    uri
  available_markets
  disc_number
  duration_ms
  explicit
  external_ids
    isrc
  external_urls
    spotify
  href
  id
  is_local
  is_playable
  name
  popularity
  preview_url
  track_number
  type
  uri


For `track_data` I'll keep:
* `album.id`
* `album.name`
* `album.release_date`
* `album.total_tracks`
* `artists.name`
* `duration_ms`
* `explicit`
* `id`
* `name`
* `popularity`

In [17]:
from datetime import date

def convert_release_date(date_str):
    if len(date_str) == 4:
        return date(int(date_str), 1, 1)
    elif len(date_str) == 7:
        return date(int(date_str[0:4]), int(date_str[5:7]), 1)
    else:
        return date(int(date_str[0:4]), int(date_str[5:7]), int(date_str[8:10]))

def get_track_data_values(track_data_entry):
    data_str = str(track_data_entry["track_data"])
    data = None
    try:
        data = ast.literal_eval(data_str)
    except:
        try:
            data = json.loads(data_str)
        except:
            print(track)
            print(data_str)
            pass
    finally:
        track_data_entry["album_id"] = data["album"]["id"]
        track_data_entry["album_name"] = data["album"]["name"]
        track_data_entry["album_release_date"] = convert_release_date(data["album"]["release_date"])
        track_data_entry["album_total_tracks"] = data["album"]["total_tracks"]
        track_data_entry["artist_names"] = list(map(lambda artist_data: artist_data["name"], data["artists"]))
        track_data_entry["duration_ms"] = data["duration_ms"]
        track_data_entry["explicit"] = data["explicit"]
        track_data_entry["id"] = data["id"]
        track_data_entry["name"] = data["name"]
        track_data_entry["popularity"] = data["popularity"]
        return track_data_entry

if not path.exists("data/processed/TrackData.csv"):
    track_data_df = stream_history_consolidated_df[stream_history_consolidated_df["track_data"].notnull()].drop(columns=["episode_data"])

    track_data_df = track_data_df.apply(get_track_data_values, axis=1).drop(columns=["track_data"])
    track_data_df.to_csv("data/processed/TrackData.csv")
else:
    track_data_df = pd.read_csv("data/processed/TrackData.csv")
    track_data_df.drop(track_data_df.columns[0], axis=1, inplace=True)

track_data_df.head()

,track_name,artist_name,counts,album_id,album_name,album_release_date,album_total_tracks,artist_names,duration_ms,explicit,id,name,popularity
0,"""61'st and Rich' It""",Thad Jones,1.0,3LbEiKB4CYtJRQLLOrpxlN,Greetings and Salutations,1975-01-01,8,"['Thad Jones', 'Mel Lewis', 'Jon Faddis', 'Swe...",495360,False,07wWm3Wkl64jUAIgyDae4I,61st and Rich'It,6
1,"""B"" Movie Box Car Blues - Live",The Blues Brothers,3.0,7nuZ830tqx4YT8jzJNpU2T,Briefcase Full of Blues,1978-11-01,12,"['The Blues Brothers', 'Joe Gastwirt']",244400,False,4jX5YH4N3E32pmLz1AlAuT,"""B"" Movie Box Car Blues - Live",38
2,'91 Maxima,Cory Wong,9.0,0A8cq3cjiMMP8lI1SeG4GN,The Optimist,2018-08-17,7,['Cory Wong'],308280,False,2JMsp6wX1S1YVDDmMoVcCx,'91 Maxima,33
3,'Deed I Do,Blossom Dearie,1.0,6Qmn8cqmmJIjuwjXiUF4n6,Blossom Dearie (Expanded Edition),1957-04-01,17,['Blossom Dearie'],133666,False,2dLM74n8z8UgW9NQhSbhm1,'Deed I Do,55
4,(Get Your Kicks On) Route 66,The Andrews Sisters,1.0,3GK2W9eAOQ6585VCGKvKkh,Their Greatest Hits And Finest Performances,1996-01-01,60,"['The Andrews Sisters', 'Bing Crosby', 'Vic Sc...",195373,False,3mNWkPhQNwGFUeJRLfXQ5Z,(Get Your Kicks On) Route 66,38


In [18]:
print("Episode data:")
print_nested_keys(ast.literal_eval(episode_data), "  ")

Episode data:
  audio_preview_url
  description
  duration_ms
  explicit
  external_urls
    spotify
  href
  id
  images
    height
    url
    width
  is_externally_hosted
  is_playable
  language
  languages
  name
  release_date
  release_date_precision
  type
  uri


For `episode_data` I'll keep:
* `description`
* `duration_ms`
* `explicit`
* `id`
* `name`
* `release_date`

In [19]:
def get_episode_data_values(episode_data_entry):
    data_str = episode_data_entry["episode_data"]
    data = None
    try:
        data = ast.literal_eval(data_str)
    except:
        try:
            data = json.loads(data_str)
        except:
            print(track)
            print(data_str)
            pass
    finally:
        episode_data_entry["description"] = data["description"]
        episode_data_entry["duration_ms"] = data["duration_ms"]
        episode_data_entry["explicit"] = data["explicit"]
        episode_data_entry["id"] = data["id"]
        episode_data_entry["name"] = data["name"]
        episode_data_entry["release_date"] = convert_release_date(data["release_date"])
        return episode_data_entry

if not path.exists("data/processed/EpisodeData.csv"):
    episode_data_df = stream_history_consolidated_df[stream_history_consolidated_df["episode_data"].notnull()].drop(columns=["track_data"])

    episode_data_df = episode_data_df.apply(get_episode_data_values, axis=1).drop(columns=["episode_data"])
    episode_data_df.to_csv("data/processed/EpisodeData.csv")
else:
    episode_data_df = pd.read_csv("data/processed/EpisodeData.csv")
    episode_data_df.drop(episode_data_df.columns[0], axis=1, inplace=True)

episode_data_df.head()

,track_name,artist_name,counts,description,duration_ms,explicit,id,name,release_date
0,Lab-Grown Meat: We Grill It,Science Vs,4.0,Silicon Valley seems to be constantly pumping ...,1778599,False,2l2L7L9zgP9fEEc5NqWkKy,Lab-Grown Meat: We Grill It,2020-10-08
1,"""I have a Queery about...Being Gay and Greek"" ...",Queer Queeries,2.0,"""What was the hardest part about being out in ...",2775991,True,3huD8Szyv9hspQB3JCJaYX,"""I have a Queery about...Being Gay and Greek"" ...",2020-09-13
2,#100 Friends and Blasphemers,Reply All,1.0,An online diary used by American teenagers con...,1305051,True,5yfeVKLwZLmwhbWC88pWCz,#100 Friends and Blasphemers,2017-06-29
3,#101 Minka,Reply All,1.0,A man takes on an impossible job: fixing the p...,2376072,True,1bIxMmQ9VgfPMsw6N3hbyW,#101 Minka,2017-07-13
4,#102 Long Distance,Reply All,1.0,"This week, a telephone scammer makes a terribl...",2759027,True,6wOtFK4x7E8N5VdOV7ejRy,#102 Long Distance,2017-07-27


Almost there, let's go through and check to make sure all of our track data is fixed. We're checking for 3 things:
* If the name provided by Spotify in initial data is not equal to name found from Spotify search (we'll probably get a lot of false positives with this, that's okay)
* If the artist provided by Spotify in initial data is not included in the artists found from the Spotify search
* If the id gotten from the Spotify search is duplicated with another track (this is sometimes okay, case-by-case basis)

In [20]:
if not path.exists("data/processed/IncorrectTracksData.csv"):
    # This means the data has not been scrubbed and needs to be.
    temp_df = pd.concat([track_data_df[track_data_df["track_name"] != track_data_df["name"]],
                         track_data_df[track_data_df.apply(lambda track: not track["artist_name"] in track["artist_names"], axis=1)],
                         track_data_df[track_data_df["id"].duplicated()]]).set_index(["track_name", "artist_name"])
    incorrect_tracks_df = temp_df.loc[~temp_df.index.duplicated(keep='last')].reset_index()
    incorrect_tracks_df[incorrect_tracks_df["artist_name"] == "Elton John"]

    incorrect_tracks_df.to_csv("data/processed/IncorrectTracksData.csv");
    print("CLEAN DATA AND RETURN TO MOVE FORWARD")
else:
    incorrect_tracks_df = pd.read_csv("data/processed/IncorrectTracksData.csv")
    incorrect_tracks_df.drop(incorrect_tracks_df.columns[0], axis=1, inplace=True)
    not_fixed_data = incorrect_tracks_df[incorrect_tracks_df["new_id"].isna()]
    if not_fixed_data.shape[0] > 0:
        print(str.format("{} more entries to fix", not_fixed_data.shape[0]))
        display(not_fixed_data)
    else:
        print("Data cleaned! Go ahead and continue!")

Data cleaned! Go ahead and continue!


In [21]:
def fix_incorrect_tracks(incorrect_tracks_new_ids):
    incorrect_tracks_fixed_data = []
    base_url = "https://api.spotify.com/v1/tracks"
    sublist_length = 50 # tracks API call only allows for 50 ids at a time
    for i in range((len(incorrect_tracks_new_ids) // sublist_length) + 1):
        new_ids = ",".join(incorrect_tracks_new_ids[i * sublist_length:(i + 1) * sublist_length])
        query_obj = {"ids": new_ids}
        res = requests.get(base_url + "?" + urlencode(query_obj), headers={
            "Authorization": "Bearer " + token
        })
        if res.status_code != 200:
            raise Exception("Spotify call failed, something has gone wrong")
        res_json = res.json()
        incorrect_tracks_fixed_data += res_json["tracks"]
    return incorrect_tracks_fixed_data

if not path.exists("data/processed/IncorrectTracksDataFixed.csv"):
    fixed_incorrect_tracks_df = incorrect_tracks_df[["track_name", "artist_name", "counts", "new_id"]]
    incorrect_tracks_new_ids = list(fixed_incorrect_tracks_df["new_id"])

    incorrect_tracks_fixed_data = fix_incorrect_tracks(incorrect_tracks_new_ids)
    fixed_incorrect_tracks_df["track_data"] = pd.Series(incorrect_tracks_fixed_data, index=fixed_incorrect_tracks_df.index)
    fixed_incorrect_tracks_df = fixed_incorrect_tracks_df.apply(get_track_data_values, axis=1).drop(columns=["track_data"])

    need_to_be_fixed = fixed_incorrect_tracks_df[fixed_incorrect_tracks_df["new_id"] != fixed_incorrect_tracks_df["id"]]
    if need_to_be_fixed.shape[0] > 0:
        print("IDS DON'T EQUAL NEW IDS:")
        display(need_to_be_fixed)
    else:
        fixed_incorrect_tracks_df.to_csv("data/processed/IncorrectTracksDataFixed.csv");
        fixed_incorrect_tracks_df.drop(columns=["new_id"], inplace=True)
        display(fixed_incorrect_tracks_df.head())
else:
    fixed_incorrect_tracks_df = pd.read_csv("data/processed/IncorrectTracksDataFixed.csv").drop(columns=["new_id"])
    fixed_incorrect_tracks_df.drop(fixed_incorrect_tracks_df.columns[0], axis=1, inplace=True)
    display(fixed_incorrect_tracks_df.head())

,track_name,artist_name,counts,album_id,album_name,album_release_date,album_total_tracks,artist_names,duration_ms,explicit,id,name,popularity
0,Cinnamon Girl,[dunkelbunt],1,3lH4gaKdQ12d4OewBkaVWl,Cinnamon Girl,2009-04-06,5,"['[dunkelbunt]', 'Boban i Marko Marcovic Orkes...",240800,False,58x6LBD5b6Tu7IPsYL99oG,Cinnamon Girl,41
1,Voulez-Vous,ABBA,1,0zdZSyxWaYmaRMPeUHcG1K,Voulez-Vous,1979-01-01,13,['ABBA'],309160,False,6jJYlWqSUHXoX15nGk3Yfe,Voulez-Vous,54
2,Dixieland Delight,Alabama,2,0eMs6LyA1bTZm8Mx8FPshM,Closer You Get,1983-01-01,10,['Alabama'],322093,False,0bM1z18RQpr61UuI1LemIK,Dixieland Delight,63
3,Dixieland Delight - Single Edit,Alabama,7,1DE97Q1jJABKEbOi8CzLSu,The Essential Alabama,2005-05-17,44,['Alabama'],236426,False,1hh3iFX5vVsiD9LMwUlQeh,Dixieland Delight - Single Edit,63
4,Boys & Girls,Alabama Shakes,3,4eOGRVKyCsVfaXysnHz0k2,Boys & Girls,2012-04-10,11,['Alabama Shakes'],205986,False,1A2hmJG6PSP92pj42MFm4W,Boys & Girls,39


Finished! Now we can merge it back into our `track_data` dataframe:

In [22]:
temp_df = pd.concat([track_data_df, fixed_incorrect_tracks_df]).set_index(["track_name", "artist_name"])
track_data_df = temp_df.loc[~temp_df.index.duplicated(keep='last')]
track_data_df.reset_index(inplace=True)

PHEW, data should be nice and clean now (_that_ took a while (about 5 days to feel comfortable with the state of the data)). Now that our tracks are cleaned, we'll transfer that data back to the main `stream_history_df` dataframe:

In [23]:
if not path.exists("data/processed/NewTracksData.csv"):
    complete_stream_history_df = stream_history_df.merge(track_data_df, on=["artist_name", "track_name"])
    complete_stream_history_df.drop(columns=["counts"], inplace=True) # Not sure why I ever added this, this is a very misleading metric the way I implemented it
    complete_stream_history_df.to_csv("data/processed/NewTracksData.csv")
else:
    complete_stream_history_df = pd.read_csv("data/processed/NewTracksData.csv")
    complete_stream_history_df.drop(complete_stream_history_df.columns[0], axis=1, inplace=True)

complete_stream_history_df.head()

,artist_name,track_name,ms_played,date,album_id,album_name,album_release_date,album_total_tracks,artist_names,duration_ms,explicit,id,name,popularity
0,Mingus Big Band,Haitian Fight Song,680,2020-01-01 00:51:00,0Gwu5X7W1mrkSTk2uZ25cv,Blues & Politics,1999-06-29,8,['Mingus Big Band'],499826,False,4hns23kYYZg0BhDwXeDxB1,Haitian Fight Song,31
1,Andy Martin & Vic Lewis,Everything You Is,3282,2020-01-01 00:52:00,7By1lfK4fTIs2YsMvA0FWH,The Project,2004-01-01,10,['Andy Martin & Vic Lewis'],459106,False,3sLmks6fCY40bBSGDjU4FO,Everything You Is,0
2,Guy Lombardo & His Royal Canadians,Auld Lang Syne,66171,2020-01-01 00:52:00,3f22Ap0VSZYWsqrGcphUnY,Christmas Classics,2004-01-01,16,['Guy Lombardo & His Royal Canadians'],128000,False,4Saza06xljloZwotqXdNle,Auld Lang Syne,31
3,Thad Jones,To You,1496,2020-01-01 00:52:00,5gfrrR8BnDgFhqGWcQaWFe,And the Danish Radio Big Band & Eclipse,2013-05-10,22,['Thad Jones'],255466,False,2408a07TNDga6lMlaIFLEU,To You,7
4,Count Basie,All Of Me,150773,2020-01-01 00:55:00,2kAN1sZjSQQDkusyXyngep,Frankly Basie / Count Basie Plays The Hits Of ...,1963-01-01,15,['Count Basie'],150773,False,0rupt7DuLo3WGecL3cyi19,All Of Me,15


## Complete!

Now we can move along to the fun stuff... the ANALYSIS

For the future: I should probably finish cleaning the Podcast data as well